<a href="https://colab.research.google.com/github/milesfking/2020-presidential-election-model/blob/main/NBA_Sentiment_Reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install and Load Packages

In [ ]:
pip install asyncpraw

In [ ]:
pip install -q transformers

In [52]:
# Import libaries
import requests
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from datetime import datetime

import asyncpraw
from asyncpraw.models import MoreComments

## Scrape Reddit

In [19]:
reddit = asyncpraw.Reddit(
    client_id="sKXeo22ih_keYkCKCeW2iw",
    client_secret="84Kps65uhBOVWk36Ci1yBnME6rBGDw",
    user_agent="mac:statpad:v1.1.1 (by /u/statpadders)",
)

In [20]:
text = []
subreddit = await reddit.subreddit("nba")
async for submission in subreddit.hot(limit=1):
    print(submission.title)
    top_level_comments = await submission.comments()
    all_comments = await submission.comments.list()
    for comment in all_comments:
      print(comment.body)

Daily Discussion Thread + Game Thread Index
Scott Foster needs some help y'all. This Lakers squad ain't it.
What’s up with everyone saying no one would watch the finals if it’s nuggets vs heat? Sounds way better than Celtics vs lakers honestly. I’d rather watch two really good teams that play with some intensity and heart than a Celtics and lakers boring matchup. Just sounds like 48 minutes of flopping and complaining for fouls.
Nuggets are going to the finals today
Jamal Murray- The Kingslayer
Just got the latest draft of the NBA script. Both the Lakers and Celtics are going to advance to the finals after being the first 2 teams to ever win a series after being down 0-3. The Lakers will win the finals in 7 after LeBron scores an NBA record 102 points including a series-winning buzzer beater 3.

Incredible.
Go on ESPN and all you see are these "analysts" talking about how bad the Celtics were last night. No mention on how dominant the Heat were and how historic their run is. I'm tired 

https://medium.com/analytics-vidhya/scraping-reddit-using-python-reddit-api-wrapper-praw-5c275e34a8f4

https://praw.readthedocs.io/en/stable/getting_started/quick_start.html

In [55]:
posts = pd.DataFrame(columns=["Title", "Text", "Comments", "Time", "Upvotes"])
subreddit = await reddit.subreddit("nba")
async for submission in subreddit.hot(limit=2):
    title = submission.title
    text = submission.selftext
    comments = []
    time = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')
    upvotes = submission.score
    top_level_comments = await submission.comments()
    for top_level_comment in submission.comments[1:]:
      if isinstance(top_level_comment, MoreComments):
        continue
    comments.append(top_level_comment.body)
    posts.loc[len(posts.index)] = [title, text, comments, time, upvotes]

In [56]:
posts

,Title,Text,Comments,Time,Upvotes
0,Daily Discussion Thread + Game Thread Index,"# Game Threads Index (May 22, 2023):\n\n|Tip-o...",[Austin Reaves was out there kinda looking lik...,2023-05-22 19:00:04,76
1,[SERIOUS NEXT DAY THREAD] Post-Game Discussion...,"Here is a place to have in depth, x's and o's,...",[\n\n**Nuggets @ Lakers**\n\n[](/DEN) **113 - ...,2023-05-23 12:00:02,33


## Perform Sentiment Analysis

In [23]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [31]:
specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
specific_model("Dillon Brooks sucks - welcome to China buddy")

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


[{'label': 'NEG', 'score': 0.9621661305427551}]